In [1]:
import pandas as pd

In [31]:
df = pd.read_excel(r"C:\Users\Divya\OneDrive\Documents\Company_name.xlsx")
df.head()

,Company Name
0,Nitori Holdings
1,Kawasaki Heavy Industries
2,Idemitsu Kosan Co.
3,Eisai
4,Fujikura


In [32]:
df.shape
df.iloc[300]

Company Name    Hiscox
Name: 300, dtype: object

In [36]:
df1 = df['Company Name'].iloc[1076:1100] # 1451 - 
# df1

In [37]:
company_list = df1.values

In [38]:
company_list[0] , len(company_list)

('Black Diamond Therapeutics Inc', 24)

In [39]:
from webdriver_manager.chrome import ChromeDriverManager
path = ChromeDriverManager().install()
path

'C:\\Users\\Divya\\.wdm\\drivers\\chromedriver\\win64\\135.0.7049.95\\chromedriver-win32/chromedriver.exe'

In [40]:
import csv
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ==== Setup ====
website = 'https://finance.yahoo.com/'
output_file = r"C:\Users\Divya\Documents\Yahoo_data92.csv"

service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)
output_data = []

# ==== Loop through each company ====
for company in company_list:
    try:
        driver.get(website)
        search_container = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@class='_yb_d7nywr _yb_owcm9n  _yb_ikgg1k finsrch-inpt  ']"))
        )
        search_container.click()
        search_container.send_keys(company)
        time.sleep(2)

        for i in range(3):
            matched_name = np.nan
            revenue_ttm = np.nan

            try:
                result_xpath = f"//ul[@role='listbox']/li[contains(@data-id , 'result-quotes-{i}')]/div[@class='modules-module_quoteLeftCol__yV9LA modules-module_Ell__N1WPm modules-module_IbBox__NPYId']"
                search_result = WebDriverWait(driver, 4).until(
                    EC.element_to_be_clickable((By.XPATH, result_xpath))
                )
                search_result.click()
                time.sleep(3)

                try:
                    matched_name = WebDriverWait(driver, 4).until(
                        EC.presence_of_element_located((By.XPATH, "//h1[@class='yf-xxbei9']"))
                    ).text.strip()
                except Exception:
                    matched_name = np.nan

                try:
                    stats_list = WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.XPATH, "//p[@class='value yf-lc8fp0']"))
                    )
                    revenue_ttm = stats_list[3].text.strip() if len(stats_list) > 3 else np.nan
                except Exception:
                    revenue_ttm = np.nan

                print(f"\n✅ {company} → {matched_name} | Revenue (TTM): {revenue_ttm}")

            except Exception as e:
                print(f"❌ Skipped result-{i+1} for '{company}': {e}")

            output_data.append({
                'Input_Company': company,
                'Matched_Company': matched_name,
                'Revenue_TTM': revenue_ttm
            })

            # Go back to search
            driver.get(website)
            search_container = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//input[@class='_yb_d7nywr _yb_owcm9n  _yb_ikgg1k finsrch-inpt  ']"))
            )
            search_container.click()
            search_container.send_keys(company)
            time.sleep(2)

    except Exception as e:
        print(f"❌ Error while processing '{company}': {e}")

driver.quit()

# ==== Save to CSV ====
df2 = pd.DataFrame(output_data)
df2.to_csv(output_file, index=False)

print(f"\n📁 CSV saved as: {output_file}")



✅ Black Diamond Therapeutics Inc → Black Diamond Therapeutics, Inc. (BDTX) | Revenue (TTM): --
❌ Skipped result-2 for 'Black Diamond Therapeutics Inc': Message: 
Stacktrace:
	GetHandleVerifier [0x00588073+60707]
	GetHandleVerifier [0x005880B4+60772]
	(No symbol) [0x003B0683]
	(No symbol) [0x003F8660]
	(No symbol) [0x003F89FB]
	(No symbol) [0x00441022]
	(No symbol) [0x0041D094]
	(No symbol) [0x0043E824]
	(No symbol) [0x0041CE46]
	(No symbol) [0x003EC5D3]
	(No symbol) [0x003ED424]
	GetHandleVerifier [0x007CBB53+2435075]
	GetHandleVerifier [0x007C70F3+2416035]
	GetHandleVerifier [0x007E349C+2531660]
	GetHandleVerifier [0x0059F145+155125]
	GetHandleVerifier [0x005A5AED+182173]
	GetHandleVerifier [0x0058F948+91640]
	GetHandleVerifier [0x0058FAF0+92064]
	GetHandleVerifier [0x0057A5B0+4704]
	BaseThreadInitThunk [0x76285D49+25]
	RtlInitializeExceptionChain [0x7798CF0B+107]
	RtlGetAppContainerNamedObjectPath [0x7798CE91+561]

❌ Skipped result-3 for 'Black Diamond Therapeutics Inc': Message: 
S

In [26]:
df = pd.DataFrame(output_data)
df.to_csv(output_file, index=False)

In [29]:
len(df)

142

In [27]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

def convert_notation_to_number(value):
    try:
        if value.endswith('B'):
            return float(value[:-1]) * 1e9
        elif value.endswith('M'):
            return float(value[:-1]) * 1e6
        elif value.endswith('K'):
            return float(value[:-1]) * 1e3
        else:
            return float(value)
    except:
        return None

# Load reference DataFrame
df_reference = pd.read_csv("visible_alpha_surprises.csv")  # Contains 'Company Name', 'Ticker', 'v'

website = "https://finance.yahoo.com/"

service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)
driver.get(website)

results = []

for company in company_list:
    try:
        search_container = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@class='_yb_d7nywr _yb_owcm9n  _yb_ikgg1k finsrch-inpt  ']"))
        )
        search_container.clear()
        search_container.send_keys(company)
        time.sleep(2)

        search_result_xpath = f"//ul[@role='listbox']/li[contains(@data-id , 'result-quotes-0')]/div[@class='modules-module_quoteLeftCol__yV9LA modules-module_Ell__N1WPm modules-module_IbBox__NPYId']"
        search_results_container = WebDriverWait(driver, 4).until(
            EC.element_to_be_clickable((By.XPATH, search_result_xpath))
        )
        search_results_container.click()
        time.sleep(3)

        full_name_text = WebDriverWait(driver, 4).until(
            EC.presence_of_element_located((By.XPATH, "//h1[@class='yf-xxbei9']"))
        ).text.strip()

        # Extract full name and ticker
        if "(" in full_name_text and ")" in full_name_text:
            company_name = full_name_text.split(" (")[0]
            extracted_ticker = full_name_text.split("(")[1].replace(")", "").strip()
        else:
            company_name = full_name_text
            extracted_ticker = None

        stats_list = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.XPATH, "//p[@class='value yf-lc8fp0']"))
        )
        revenue_ttm_text = stats_list[3].text.strip()
        revenue_ttm_num = convert_notation_to_number(revenue_ttm_text)

        # Try to match revenue and fetch correct ticker from reference dataframe
        matched_ticker = None
        if revenue_ttm_num:
            match_row = df_reference[df_reference["v"].apply(lambda x: abs(x - revenue_ttm_num) < 1e6)]
            if not match_row.empty:
                matched_ticker = match_row.iloc[0]["ticker"]

        results.append({
            "Company Name": company_name,
            "Extracted Ticker": extracted_ticker,
            "Revenue (TTM)": revenue_ttm_text,
            "Revenue Num": revenue_ttm_num,
            "Matched Ticker": matched_ticker
        })

        # Go back to search page
        driver.get(website)

    except Exception as e:
        print(f"❌ Error while processing '{company}': {e}")
        results.append({
            "Company Name": company,
            "Extracted Ticker": None,
            "Revenue (TTM)": None,
            "Revenue Num": None,
            "Matched Ticker": None
        })
        driver.get(website)

# Quit driver
driver.quit()

# Save final DataFrame
df_final = pd.DataFrame(results)
df_final.to_csv("yahoo_finance_results.csv", index=False)
print("✅ Saved to yahoo_finance_results.csv")


❌ Error while processing 'Keysight Technologies, Inc.': Message: 
Stacktrace:
	GetHandleVerifier [0x003F80E3+60707]
	GetHandleVerifier [0x003F8124+60772]
	(No symbol) [0x00220683]
	(No symbol) [0x00268660]
	(No symbol) [0x002689FB]
	(No symbol) [0x002B1022]
	(No symbol) [0x0028D094]
	(No symbol) [0x002AE824]
	(No symbol) [0x0028CE46]
	(No symbol) [0x0025C5D3]
	(No symbol) [0x0025D424]
	GetHandleVerifier [0x0063BBC3+2435075]
	GetHandleVerifier [0x00637163+2416035]
	GetHandleVerifier [0x0065350C+2531660]
	GetHandleVerifier [0x0040F1B5+155125]
	GetHandleVerifier [0x00415B5D+182173]
	GetHandleVerifier [0x003FF9B8+91640]
	GetHandleVerifier [0x003FFB60+92064]
	GetHandleVerifier [0x003EA620+4704]
	BaseThreadInitThunk [0x779B5D49+25]
	RtlInitializeExceptionChain [0x77E2CF0B+107]
	RtlGetAppContainerNamedObjectPath [0x77E2CE91+561]

✅ Saved to yahoo_finance_results.csv
